In [ ]:
#@title Setup inicial
#@markdown Es necesario ejecutar esta celda en primer lugar para instalar los softwares necesarios para descargar y preprocesar los datos de RNA-seq

from google.colab import files

#Make directorie to store raw counts
!mkdir fastqs

#Download FastQC for quality control
!wget https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.9.zip
!unzip fastqc_v0.11.9.zip
%cd FastQC
!chmod 755 fastqc
%cd ..
!mkdir fastqc

#Download BBTools for trimming and preprocessing raw counts
!wget -O BBMap_38.82.tar.gz 'https://downloads.sourceforge.net/project/bbmap/BBMap_38.82.tar.gz?r=https%3A%2F%2Fsourceforge.net%2Fprojects%2Fbbmap%2Ffiles%2FBBMap_38.82.tar.gz%2Fdownload&ts=1587572478'
!tar xvf BBMap_38.82.tar.gz
!mkdir trimmed

#Download Kallisto for counting transcripts abundance
!wget https://github.com/pachterlab/kallisto/releases/download/v0.46.2/kallisto_linux-v0.46.2.tar.gz
!tar zxf kallisto_linux-v0.46.2.tar.gz
!mv kallisto kallisto-v0.46.2
!mkdir kallisto

#Downloading Reference transcriptome
!wget http://www.strawberryblast.ml:8080/strawberry/docs/annotation/Fragaria_ananassa_v1.0.a2.transcripts.fa.gz
!gzip -d Fragaria_ananassa_v1.0.a2.transcripts.fa.gz

#Clean transcript headers
!sed 's/ gene=.*//' Fragaria_ananassa_v1.0.a2.transcripts.fa > Fragaria_ananassa_v1.0.a2.transcripts.clean.fa

#Creating kallisto index
!/content/kallisto-v0.46.2/kallisto index -i Fragaria_ananassa_v1.0.a2_kallisto-0.46.2 Fragaria_ananassa_v1.0.a2.transcripts.clean.fa
googledrive=False


In [ ]:
#@title Goodle drive
#@markdown Ejecutar esta celda en caso de que queramos guardar los archivos finales en google drive en lugar de descargarlos directamente (no muy recomendado ya que los archivos finales pueden ocupar bastante espacio en drive).

from google.colab import drive
drive.mount('/content/drive')
googledrive = True

In [ ]:
#@title White achene
#@markdown Ejecutar esta celda para descargar los archivos de aquenios en estadio blanco

#Limpiamos los directorios individiduales en caso de que hayamos descargado los datos de otro
#tejido para economizar el espacio en disco

!cd fastqs ; rm *
!cd fastqc ; rm -rf *
!cd trimmed ; rm *
!cd kallisto ; rm -rf *

runname=["ERR1817341","ERR1817342",
         "ERR1817343","ERR1817344"]

#Descargamos los archivos

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/001/ERR1817341/ERR1817341_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/001/ERR1817341/ERR1817341_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/002/ERR1817342/ERR1817342_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/002/ERR1817342/ERR1817342_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/003/ERR1817343/ERR1817343_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/003/ERR1817343/ERR1817343_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/004/ERR1817344/ERR1817344_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/004/ERR1817344/ERR1817344_2.fastq.gz

#Quality control

for id in runname:
  !/content/FastQC/fastqc -t 2 --extract -o fastqc fastqs/{id}_1.fastq.gz fastqs/{id}_2.fastq.gz

#Trimming

for id in runname:
  !/content/bbmap/bbduk.sh threads=2 in1=fastqs/{id}_1.fastq.gz in2=fastqs/{id}_2.fastq.gz out1=trimmed/{id}_1.fastq.gz out2=trimmed/{id}_2.fastq.gz ref=/content/bbmap/resources/adapters.fa tbo tpe ktrim=r k=21 mink=9 hdist=1 qtrim=rl trimq=15 minlength=36 maxns=1 2> trimmed/{id}.log

runname=["ERR1817341","ERR1817342",
         "ERR1817343","ERR1817344"]

#Quantification

for id in runname:
  !/content/kallisto-v0.46.2/kallisto quant -t 2 -i /content/Fragaria_ananassa_v1.0.a2_kallisto-0.46.2 --rf-stranded -b 100 -o kallisto/{id} trimmed/{id}_1.fastq.gz trimmed/{id}_2.fastq.gz

#Comprimimos y descargamos los resultados

!cd fastqc; zip -q -r ../WA_QC *
!cd kallisto; zip -q -r ../WA_kallisto *

if googledrive:
  shutil.copy(f'WA_QC.zip', f"/content/drive/MyDrive/colab")
  shutil.copy(f'WA_kallisto.zip', f"/content/drive/MyDrive/colab")
else:
  files.download(f'WA_QC.zip')
  files.download(f'WA_kallisto.zip')




In [ ]:
#@title Red achene
#@markdown Ejecutar esta celda para descargar los archivos de aquenios en estadio rojo

#Limpiamos los directorios individiduales en caso de que hayamos descargado los datos de otro
#tejido para economizar el espacio en disco

!cd fastqs ; rm *
!cd fastqc ; rm -rf *
!cd trimmed ; rm *
!cd kallisto ; rm -rf *

runname=["ERR1817339","ERR1817340","ERR1817346","ERR1817347","ERR1817351","ERR1817352","ERR1817357","ERR1817359","ERR1817365","ERR1817366","ERR1817373","ERR1817374","ERR1817378","ERR1817380"]
runname=["ERR1817345","ERR1817348",
         "ERR1817349","ERR1817350"]

#Descargamos los archivos

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/005/ERR1817345/ERR1817345_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/005/ERR1817345/ERR1817345_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/008/ERR1817348/ERR1817348_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/008/ERR1817348/ERR1817348_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/009/ERR1817349/ERR1817349_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/009/ERR1817349/ERR1817349_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/000/ERR1817350/ERR1817350_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/000/ERR1817350/ERR1817350_2.fastq.gz

#Quality control

for id in runname:
  !/content/FastQC/fastqc -t 2 --extract -o fastqc fastqs/{id}_1.fastq.gz fastqs/{id}_2.fastq.gz

#Trimming

for id in runname:
  !/content/bbmap/bbduk.sh threads=2 in1=fastqs/{id}_1.fastq.gz in2=fastqs/{id}_2.fastq.gz out1=trimmed/{id}_1.fastq.gz out2=trimmed/{id}_2.fastq.gz ref=/content/bbmap/resources/adapters.fa tbo tpe ktrim=r k=21 mink=9 hdist=1 qtrim=rl trimq=15 minlength=36 maxns=1 2> trimmed/{id}.log

#Quantification

for id in runname:
  !/content/kallisto-v0.46.2/kallisto quant -t 2 -i /content/Fragaria_ananassa_v1.0.a2_kallisto-0.46.2 --rf-stranded -b 100 -o kallisto/{id} trimmed/{id}_1.fastq.gz trimmed/{id}_2.fastq.gz

#Comprimimos y descargamos los resultados

!cd fastqc; zip -q -r ../RA_QC *
!cd kallisto; zip -q -r ../RA_kallisto *

if googledrive:
  shutil.copy(f'RA_QC.zip', f"/content/drive/MyDrive/colab")
  shutil.copy(f'RA_kallisto.zip', f"/content/drive/MyDrive/colab")
else:
  files.download(f'RA_QC.zip')
  files.download(f'RA_kallisto.zip')

In [ ]:
#@title Turning achene
#@markdown Ejecutar esta celda para descargar los archivos de aquenios en estadio rosa

#Limpiamos los directorios individiduales en caso de que hayamos descargado los datos de otro
#tejido para economizar el espacio en disco

!cd fastqs ; rm *
!cd fastqc ; rm -rf *
!cd trimmed ; rm *
!cd kallisto ; rm -rf *

runname=["ERR1817353","ERR1817354",
         "ERR1817355","ERR1817356"]

#Descargamos los archivos

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/003/ERR1817353/ERR1817353_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/003/ERR1817353/ERR1817353_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/004/ERR1817354/ERR1817354_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/004/ERR1817354/ERR1817354_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/005/ERR1817355/ERR1817355_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/005/ERR1817355/ERR1817355_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/006/ERR1817356/ERR1817356_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/006/ERR1817356/ERR1817356_2.fastq.gz

#Quality control

for id in runname:
  !/content/FastQC/fastqc -t 2 --extract -o fastqc fastqs/{id}_1.fastq.gz fastqs/{id}_2.fastq.gz

#Trimming

for id in runname:
  !/content/bbmap/bbduk.sh threads=2 in1=fastqs/{id}_1.fastq.gz in2=fastqs/{id}_2.fastq.gz out1=trimmed/{id}_1.fastq.gz out2=trimmed/{id}_2.fastq.gz ref=/content/bbmap/resources/adapters.fa tbo tpe ktrim=r k=21 mink=9 hdist=1 qtrim=rl trimq=15 minlength=36 maxns=1 2> trimmed/{id}.log

#Quantification

for id in runname:
  !/content/kallisto-v0.46.2/kallisto quant -t 2 -i /content/Fragaria_ananassa_v1.0.a2_kallisto-0.46.2 --rf-stranded -b 100 -o kallisto/{id} trimmed/{id}_1.fastq.gz trimmed/{id}_2.fastq.gz

#Comprimimos y descargamos los resultados

!cd fastqc; zip -q -r ../TA_QC *
!cd kallisto; zip -q -r ../TA_kallisto *

if googledrive:
  shutil.copy(f'TA_QC.zip', f"/content/drive/MyDrive/colab")
  shutil.copy(f'TA_kallisto.zip', f"/content/drive/MyDrive/colab")
else:
  files.download(f'TA_QC.zip')
  files.download(f'TA_kallisto.zip')

In [ ]:
#@title Green achene
#@markdown Ejecutar esta celda para descargar los archivos de aquenios en estadio verde

#Limpiamos los directorios individiduales en caso de que hayamos descargado los datos de otro
#tejido para economizar el espacio en disco

!cd fastqs ; rm *
!cd fastqc ; rm -rf *
!cd trimmed ; rm *
!cd kallisto ; rm -rf *

runname=["ERR1817358","ERR1817360",
         "ERR1817361","ERR1817362"]

#Descargamos los archivos

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/008/ERR1817358/ERR1817358_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/008/ERR1817358/ERR1817358_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/000/ERR1817360/ERR1817360_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/000/ERR1817360/ERR1817360_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/001/ERR1817361/ERR1817361_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/001/ERR1817361/ERR1817361_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/002/ERR1817362/ERR1817362_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/002/ERR1817362/ERR1817362_2.fastq.gz

#Quality control

for id in runname:
  !/content/FastQC/fastqc -t 2 --extract -o fastqc fastqs/{id}_1.fastq.gz fastqs/{id}_2.fastq.gz

#Trimming

for id in runname:
  !/content/bbmap/bbduk.sh threads=2 in1=fastqs/{id}_1.fastq.gz in2=fastqs/{id}_2.fastq.gz out1=trimmed/{id}_1.fastq.gz out2=trimmed/{id}_2.fastq.gz ref=/content/bbmap/resources/adapters.fa tbo tpe ktrim=r k=21 mink=9 hdist=1 qtrim=rl trimq=15 minlength=36 maxns=1 2> trimmed/{id}.log

#Quantification

for id in runname:
  !/content/kallisto-v0.46.2/kallisto quant -t 2 -i /content/Fragaria_ananassa_v1.0.a2_kallisto-0.46.2 --rf-stranded -b 100 -o kallisto/{id} trimmed/{id}_1.fastq.gz trimmed/{id}_2.fastq.gz

#Comprimimos y descargamos los resultados

!cd fastqc; zip -q -r ../GA_QC *
!cd kallisto; zip -q -r ../GA_kallisto *

if googledrive:
  shutil.copy(f'GA_QC.zip', f"/content/drive/MyDrive/colab")
  shutil.copy(f'GA_kallisto.zip', f"/content/drive/MyDrive/colab")
else:
  files.download(f'GA_QC.zip')
  files.download(f'GA_kallisto.zip')

In [ ]:
#@title Leaf
#@markdown Ejecutar esta celda para descargar los archivos de hojas.
#@markdown Estos datos no fueron utilizados al final pero en caso de que fuesen necesarios simplemente habría que ejecutar esta celda

#Limpiamos los directorios individiduales en caso de que hayamos descargado los datos de otro
#tejido para economizar el espacio en disco

!cd fastqs ; rm *
!cd fastqc ; rm -rf *
!cd trimmed ; rm *
!cd kallisto ; rm -rf *

runname=["ERR1817363","ERR1817364"]

#Descargamos los archivos
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/003/ERR1817363/ERR1817363_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/003/ERR1817363/ERR1817363_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/004/ERR1817364/ERR1817364_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/004/ERR1817364/ERR1817364_2.fastq.gz


#Quality control

for id in runname:
  !/content/FastQC/fastqc -t 2 --extract -o fastqc fastqs/{id}_1.fastq.gz fastqs/{id}_2.fastq.gz

#Trimming

for id in runname:
  !/content/bbmap/bbduk.sh threads=2 in1=fastqs/{id}_1.fastq.gz in2=fastqs/{id}_2.fastq.gz out1=trimmed/{id}_1.fastq.gz out2=trimmed/{id}_2.fastq.gz ref=/content/bbmap/resources/adapters.fa tbo tpe ktrim=r k=21 mink=9 hdist=1 qtrim=rl trimq=15 minlength=36 maxns=1 2> trimmed/{id}.log

#Quantification

for id in runname:
  !/content/kallisto-v0.46.2/kallisto quant -t 2 -i /content/Fragaria_ananassa_v1.0.a2_kallisto-0.46.2 --rf-stranded -b 100 -o kallisto/{id} trimmed/{id}_1.fastq.gz trimmed/{id}_2.fastq.gz

#Comprimimos y descargamos los resultados

#!cd fastqc; zip -q -r ../L_QC *
!cd kallisto; zip -q -r ../L_kallisto *

if googledrive:
  #shutil.copy(f'L_QC.zip', f"/content/drive/MyDrive/colab")
  shutil.copy(f'L_kallisto.zip', f"/content/drive/MyDrive/colab")
else:
  #files.download(f'L_QC.zip')
  files.download(f'L_kallisto.zip')

In [ ]:
#@title Root
#@markdown Ejecutar esta celda para descargar los archivos de raíces.
#@markdown Estos datos no fueron utilizados al final pero en caso de que fuesen necesarios simplemente habría que ejecutar esta celda

#Limpiamos los directorios individiduales en caso de que hayamos descargado los datos de otro
#tejido para economizar el espacio en disco

!cd fastqs ; rm *
!cd fastqc ; rm -rf *
!cd trimmed ; rm *
!cd kallisto ; rm -rf *

runname=["ERR1817367","ERR1817368"]

#Descargamos los archivos

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/007/ERR1817367/ERR1817367_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/007/ERR1817367/ERR1817367_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/008/ERR1817368/ERR1817368_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/008/ERR1817368/ERR1817368_2.fastq.gz

#Quality control

#for id in runname:
#  !/content/FastQC/fastqc -t 2 --extract -o fastqc fastqs/{id}_1.fastq.gz fastqs/{id}_2.fastq.gz

#Trimming

for id in runname:
  !/content/bbmap/bbduk.sh threads=2 in1=fastqs/{id}_1.fastq.gz in2=fastqs/{id}_2.fastq.gz out1=trimmed/{id}_1.fastq.gz out2=trimmed/{id}_2.fastq.gz ref=/content/bbmap/resources/adapters.fa tbo tpe ktrim=r k=21 mink=9 hdist=1 qtrim=rl trimq=15 minlength=36 maxns=1 2> trimmed/{id}.log

#Quantification

for id in runname:
  !/content/kallisto-v0.46.2/kallisto quant -t 2 -i /content/Fragaria_ananassa_v1.0.a2_kallisto-0.46.2 --rf-stranded -b 100 -o kallisto/{id} trimmed/{id}_1.fastq.gz trimmed/{id}_2.fastq.gz

#Comprimimos y descargamos los resultados

#!cd fastqc; zip -q -r ../R_QC *
!cd kallisto; zip -q -r ../R_kallisto *

if googledrive:
  #shutil.copy(f'R_QC.zip', f"/content/drive/MyDrive/colab")
  shutil.copy(f'R_kallisto.zip', f"/content/drive/MyDrive/colab")
else:
  #files.download(f'R_QC.zip')
  files.download(f'R_kallisto.zip')

In [ ]:
#@title White receptacle
#@markdown Ejecutar esta celda para descargar los archivos de receptáculos en estadio blanco

#Limpiamos los directorios individiduales en caso de que hayamos descargado los datos de otro
#tejido para economizar el espacio en disco

!cd fastqs ; rm *
!cd fastqc ; rm -rf *
!cd trimmed ; rm *
!cd kallisto ; rm -rf *

runname=["ERR1817369","ERR1817370",
         "ERR1817371","ERR1817372"]

#Descargamos los archivos

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/009/ERR1817369/ERR1817369_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/009/ERR1817369/ERR1817369_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/000/ERR1817370/ERR1817370_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/000/ERR1817370/ERR1817370_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/001/ERR1817371/ERR1817371_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/001/ERR1817371/ERR1817371_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/002/ERR1817372/ERR1817372_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/002/ERR1817372/ERR1817372_2.fastq.gz

#Quality control

#for id in runname:
#  !/content/FastQC/fastqc -t 2 --extract -o fastqc fastqs/{id}_1.fastq.gz fastqs/{id}_2.fastq.gz

#Trimming

for id in runname:
  !/content/bbmap/bbduk.sh threads=2 in1=fastqs/{id}_1.fastq.gz in2=fastqs/{id}_2.fastq.gz out1=trimmed/{id}_1.fastq.gz out2=trimmed/{id}_2.fastq.gz ref=/content/bbmap/resources/adapters.fa tbo tpe ktrim=r k=21 mink=9 hdist=1 qtrim=rl trimq=15 minlength=36 maxns=1 2> trimmed/{id}.log

#Quantification

for id in runname:
  !/content/kallisto-v0.46.2/kallisto quant -t 2 -i /content/Fragaria_ananassa_v1.0.a2_kallisto-0.46.2 --rf-stranded -b 100 -o kallisto/{id} trimmed/{id}_1.fastq.gz trimmed/{id}_2.fastq.gz

#Comprimimos y descargamos los resultados

#!cd fastqc; zip -q -r ../WR_QC *
!cd kallisto; zip -q -r ../WR_kallisto *

if googledrive:
  #shutil.copy(f'WR_QC.zip', f"/content/drive/MyDrive/colab")
  shutil.copy(f'WR_kallisto.zip', f"/content/drive/MyDrive/colab")
else:
  #files.download(f'WR_QC.zip')
  files.download(f'WR_kallisto.zip')

In [ ]:
#@title Red receptacle
#@markdown Ejecutar esta celda para descargar los archivos de receptaculos en estadio rojo

#Limpiamos los directorios individiduales en caso de que hayamos descargado los datos de otro
#tejido para economizar el espacio en disco

!cd fastqs ; rm *
!cd fastqc ; rm -rf *
!cd trimmed ; rm *
!cd kallisto ; rm -rf *

runname=["ERR1817375","ERR1817376",
         "ERR1817377","ERR1817379"]

#Descargamos los archivos

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/005/ERR1817375/ERR1817375_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/005/ERR1817375/ERR1817375_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/006/ERR1817376/ERR1817376_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/006/ERR1817376/ERR1817376_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/007/ERR1817377/ERR1817377_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/007/ERR1817377/ERR1817377_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/009/ERR1817379/ERR1817379_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/009/ERR1817379/ERR1817379_2.fastq.gz

#Quality control

#for id in runname:
#  !/content/FastQC/fastqc -t 2 --extract -o fastqc fastqs/{id}_1.fastq.gz fastqs/{id}_2.fastq.gz

#Trimming

for id in runname:
  !/content/bbmap/bbduk.sh threads=2 in1=fastqs/{id}_1.fastq.gz in2=fastqs/{id}_2.fastq.gz out1=trimmed/{id}_1.fastq.gz out2=trimmed/{id}_2.fastq.gz ref=/content/bbmap/resources/adapters.fa tbo tpe ktrim=r k=21 mink=9 hdist=1 qtrim=rl trimq=15 minlength=36 maxns=1 2> trimmed/{id}.log

#Quantification

for id in runname:
  !/content/kallisto-v0.46.2/kallisto quant -t 2 -i /content/Fragaria_ananassa_v1.0.a2_kallisto-0.46.2 --rf-stranded -b 100 -o kallisto/{id} trimmed/{id}_1.fastq.gz trimmed/{id}_2.fastq.gz

#Comprimimos y descargamos los resultados

#!cd fastqc; zip -q -r ../RR_QC *
!cd kallisto; zip -q -r ../RR_kallisto *

if googledrive:
  #shutil.copy(f'RR_QC.zip', f"/content/drive/MyDrive/colab")
  shutil.copy(f'RR_kallisto.zip', f"/content/drive/MyDrive/colab")
else:
  #files.download(f'RR_QC.zip')
  files.download(f'RR_kallisto.zip')

In [ ]:
#@title Turning receptacle
#@markdown Ejecutar esta celda para descargar los archivos de receptaculos en estadio rosa

#Limpiamos los directorios individiduales en caso de que hayamos descargado los datos de otro
#tejido para economizar el espacio en disco

!cd fastqs ; rm *
!cd fastqc ; rm -rf *
!cd trimmed ; rm *
!cd kallisto ; rm -rf *

runname=["ERR1817381","ERR1817382",
         "ERR1817383","ERR1817386"]

#Descargamos los archivos

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/001/ERR1817381/ERR1817381_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/001/ERR1817381/ERR1817381_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/002/ERR1817382/ERR1817382_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/002/ERR1817382/ERR1817382_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/003/ERR1817383/ERR1817383_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/003/ERR1817383/ERR1817383_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/006/ERR1817386/ERR1817386_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/006/ERR1817386/ERR1817386_2.fastq.gz

#Quality control

#for id in runname:
#  !/content/FastQC/fastqc -t 2 --extract -o fastqc fastqs/{id}_1.fastq.gz fastqs/{id}_2.fastq.gz

#Trimming

for id in runname:
  !/content/bbmap/bbduk.sh threads=2 in1=fastqs/{id}_1.fastq.gz in2=fastqs/{id}_2.fastq.gz out1=trimmed/{id}_1.fastq.gz out2=trimmed/{id}_2.fastq.gz ref=/content/bbmap/resources/adapters.fa tbo tpe ktrim=r k=21 mink=9 hdist=1 qtrim=rl trimq=15 minlength=36 maxns=1 2> trimmed/{id}.log

#Quantification

for id in runname:
  !/content/kallisto-v0.46.2/kallisto quant -t 2 -i /content/Fragaria_ananassa_v1.0.a2_kallisto-0.46.2 --rf-stranded -b 100 -o kallisto/{id} trimmed/{id}_1.fastq.gz trimmed/{id}_2.fastq.gz

#Comprimimos y descargamos los resultados

#!cd fastqc; zip -q -r ../TR_QC *
!cd kallisto; zip -q -r ../TR_kallisto *

if googledrive:
  #shutil.copy(f'TR_QC.zip', f"/content/drive/MyDrive/colab")
  shutil.copy(f'TR_kallisto.zip', f"/content/drive/MyDrive/colab")
else:
  #files.download(f'TR_QC.zip')
  files.download(f'TR_kallisto.zip')

In [ ]:
#@title Green receptacle
#@markdown Ejecutar esta celda para descargar los archivos de receptaculos en estadio verde

#Limpiamos los directorios individiduales en caso de que hayamos descargado los datos de otro
#tejido para economizar el espacio en disco

!cd fastqs ; rm *
!cd fastqc ; rm -rf *
!cd trimmed ; rm *
!cd kallisto ; rm -rf *

runname=["ERR1817387","ERR1817388",
         "ERR1817389","ERR1817390"]

#Descargamos los archivos

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/007/ERR1817387/ERR1817387_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/007/ERR1817387/ERR1817387_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/008/ERR1817388/ERR1817388_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/008/ERR1817388/ERR1817388_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/009/ERR1817389/ERR1817389_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/009/ERR1817389/ERR1817389_2.fastq.gz

!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/000/ERR1817390/ERR1817390_1.fastq.gz
!cd fastqs; wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR181/000/ERR1817390/ERR1817390_2.fastq.gz

#Quality control

#for id in runname:
#  !/content/FastQC/fastqc -t 2 --extract -o fastqc fastqs/{id}_1.fastq.gz fastqs/{id}_2.fastq.gz

#Trimming

for id in runname:
  !/content/bbmap/bbduk.sh threads=2 in1=fastqs/{id}_1.fastq.gz in2=fastqs/{id}_2.fastq.gz out1=trimmed/{id}_1.fastq.gz out2=trimmed/{id}_2.fastq.gz ref=/content/bbmap/resources/adapters.fa tbo tpe ktrim=r k=21 mink=9 hdist=1 qtrim=rl trimq=15 minlength=36 maxns=1 2> trimmed/{id}.log

#Quantification

for id in runname:
  !/content/kallisto-v0.46.2/kallisto quant -t 2 -i /content/Fragaria_ananassa_v1.0.a2_kallisto-0.46.2 --rf-stranded -b 100 -o kallisto/{id} trimmed/{id}_1.fastq.gz trimmed/{id}_2.fastq.gz

#Comprimimos y descargamos los resultados

#!cd fastqc; zip -q -r ../GR_QC *
!cd kallisto; zip -q -r ../GR_kallisto *

if googledrive:
  #shutil.copy(f'GR_QC.zip', f"/content/drive/MyDrive/colab")
  shutil.copy(f'GR_kallisto.zip', f"/content/drive/MyDrive/colab")
else:
  #files.download(f'GR_QC.zip')
  files.download(f'GR_kallisto.zip')